In [1]:
import os
import sys
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [126]:
def train(sess):
    images = tf.placeholder('float', shape=[None, 28, 28, 1], name='images')
    x = tf.reshape(images, [-1, 784])
    y_ = tf.placeholder('float', shape=[None, 10])
    w = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))

    sess.run(tf.global_variables_initializer())
    scores = tf.nn.softmax(tf.matmul(x, w) + b, name='scores')
    cross_entropy = -tf.reduce_sum(y_ * tf.log(scores))
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

    mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
    for _ in range(1000):
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    return (images, scores)

In [95]:
def export(sess, images, scores):
    model_version = 1
    export_path_base = "./tmp"
    export_path = os.path.join(tf.compat.as_bytes(export_path_base), tf.compat.as_bytes(str(model_version)))

    builder = tf.saved_model.builder.SavedModelBuilder(export_path)

    tensor_info_images = tf.saved_model.utils.build_tensor_info(images)
    tensor_info_scores = tf.saved_model.utils.build_tensor_info(scores)

    prediction_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
            inputs={'images': tensor_info_images},
            outputs={'scores': tensor_info_scores},
            method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))
    
    builder.add_meta_graph_and_variables(
        sess, [tf.saved_model.tag_constants.SERVING],
        signature_def_map={
             tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: prediction_signature,
        }
    )
    builder.save(as_text=False)

In [97]:
with tf.Session() as sess:
    images, scores = train(sess)
    export(sess, images, scores)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./tmp/1/saved_model.pb


start the container.
```
docker run -p 8503:8501 --rm --mount type=bind,source=/Users/tdual/Workspace/tf_serving/tmp,target=/models/mnist -e MODEL_NAME=mnist -t tensorflow/serving &
```

In [33]:
filename = "/Users/tdual/Workspace/tf_serving/images/" + "test_8_1170.jpg"

In [119]:
from skimage import io
img = io.imread(filename)

In [120]:
predict_request = json.dumps({"instances" : [img.tolist()]})

In [121]:
import json
import requests

In [122]:
response = requests.get("http://localhost:8503/v1/models/mnist")
response.json()

{'model_version_status': [{'state': 'AVAILABLE',
   'status': {'error_code': 'OK', 'error_message': ''},
   'version': '1'}]}

In [123]:
response = requests.get("http://localhost:8503/v1/models/mnist/metadata")
response.json()

{'metadata': {'signature_def': {'signature_def': {'serving_default': {'inputs': {'images': {'dtype': 'DT_FLOAT',
       'name': 'images_11:0',
       'tensor_shape': {'dim': [{'name': '', 'size': '-1'},
         {'name': '', 'size': '28'},
         {'name': '', 'size': '28'},
         {'name': '', 'size': '1'}],
        'unknown_rank': False}}},
     'method_name': 'tensorflow/serving/predict',
     'outputs': {'scores': {'dtype': 'DT_FLOAT',
       'name': 'scores_11:0',
       'tensor_shape': {'dim': [{'name': '', 'size': '-1'},
         {'name': '', 'size': '10'}],
        'unknown_rank': False}}}}}}},
 'model_spec': {'name': 'mnist', 'signature_name': '', 'version': '1'}}

In [124]:
url = "http://localhost:8503/v1/models/mnist:predict"

In [125]:
response = requests.post(url, data=predict_request)
response.json()

{'predictions': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]]}